In [1]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.8
# plt.rcParams['lines.markersize'] = 4.0
# plt.rcParams['figure.figsize'] = 20, 10

In [2]:
# 個別csv
csv1_path = "./csv/nod2-1_processed.csv"
csv2_path = "./csv/nod2-2_processed.csv"
csv3_path = "./csv/nod2-3_processed.csv"
csv4_path = "./csv/other2-1_processed.csv"
csv5_path = "./csv/other2-2_processed.csv"
csv6_path = "./csv/other2-3_processed.csv"

df1 = pd.read_csv(csv1_path, engine='python')
df2 = pd.read_csv(csv2_path, engine='python')
df3 = pd.read_csv(csv3_path, engine='python')
df4 = pd.read_csv(csv4_path, engine='python')
df5 = pd.read_csv(csv5_path, engine='python')
df6 = pd.read_csv(csv6_path, engine='python')

# 統合したcsv
csv_path = "./csv/entire.csv"
df = pd.read_csv(csv_path, engine='python')

In [3]:
df['label'].astype(int)
cl0 = (df['label'] == 0)
cl1 = (df['label'] == 1)
print(f"cl0: {cl0.sum()}, cl1: {cl1.sum()}")
print(f"cl0: {cl0.sum()/(cl0.sum() + cl1.sum())}, cl1: {cl1.sum()/(cl0.sum() + cl1.sum())}")

df

cl0: 223, cl1: 146
cl0: 0.6043360433604336, cl1: 0.3956639566395664


,mean_x,std_x,mad_x,max_x,min_x,energy_x,entropy_x,iqr_x,range_x,skewness_x,...,entropy_z,iqr_z,range_z,skewness_z,kurtosis_z,frame_in,frame_out,timestamp_in,timestamp_out,label
0,0.103313,0.090222,0.0660,0.273,-0.060,0.018813,3.282907,0.13100,0.333,0.030519,...,3.256259,0.01900,0.044,-0.313366,-0.941373,1.0,32.0,0.000,1.033,1.0
1,0.150000,0.065397,0.0620,0.273,0.049,0.026777,3.230602,0.12550,0.224,0.214725,...,3.376073,0.00825,0.036,-0.612509,0.289567,17.0,48.0,0.533,1.567,1.0
2,0.139531,0.066109,0.0545,0.254,0.001,0.023839,3.326393,0.10500,0.253,-0.220985,...,3.354940,0.00725,0.025,0.068811,-0.110982,33.0,64.0,1.067,2.100,1.0
3,0.185719,0.093650,0.0660,0.379,0.001,0.043262,3.316339,0.12300,0.378,0.200418,...,3.168633,0.01625,0.039,0.497027,-0.983946,49.0,80.0,1.600,2.633,1.0
4,0.250437,0.087917,0.0855,0.396,0.096,0.070448,3.282166,0.15275,0.300,0.107027,...,3.204525,0.01350,0.038,0.365520,-0.838306,65.0,96.0,2.133,3.167,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,-0.056281,0.048374,0.0270,0.050,-0.121,0.005508,3.172448,0.04550,0.171,0.776636,...,3.327885,0.05100,0.108,-0.441098,-0.889628,1153.0,1184.0,38.400,39.433,0.0
365,0.002250,0.037818,0.0220,0.050,-0.060,0.001435,3.204886,0.07150,0.110,-0.577385,...,3.170101,0.10300,0.144,-0.086159,-1.484037,1169.0,1200.0,38.933,39.967,0.0
366,0.034281,0.013008,0.0080,0.051,0.008,0.001344,3.301656,0.01725,0.043,-0.660231,...,3.149270,0.01750,0.093,0.983736,0.145093,1185.0,1216.0,39.467,40.500,0.0
367,0.039656,0.006826,0.0045,0.051,0.028,0.001619,3.242756,0.00925,0.023,-0.250682,...,2.854914,0.01950,0.037,0.905031,-0.782166,1201.0,1232.0,40.000,41.033,0.0


In [4]:
df = df.replace([np.inf, -np.inf], np.nan)
print(f"欠損値:\n{df.isnull().sum()}")
data = df.dropna(axis=1, how="any")

X = data.iloc[:, :-5]
y = data["label"].astype(int)

欠損値:
mean_x           0
std_x            0
mad_x            0
max_x            0
min_x            0
energy_x         0
entropy_x        0
iqr_x            0
range_x          0
skewness_x       0
kurtosis_x       0
mean_y           0
std_y            0
mad_y            0
max_y            0
min_y            0
energy_y         0
entropy_y        0
iqr_y            0
range_y          0
skewness_y       0
kurtosis_y       0
mean_z           0
std_z            0
mad_z            0
max_z            0
min_z            0
energy_z         0
entropy_z        0
iqr_z            0
range_z          0
skewness_z       0
kurtosis_z       0
frame_in         0
frame_out        0
timestamp_in     0
timestamp_out    0
label            0
dtype: int64


In [5]:
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

# RandomForestRegressorでBorutaを実行
rf = RandomForestRegressor(n_jobs=-1, max_depth=5)
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=1)
feat_selector.fit(X.values, y.values)

# 選択された特徴量を確認
selected = feat_selector.support_
print('選択された特徴量の数: %d' % np.sum(selected))
print(selected)
print(X.columns[selected])

X_selected = X.iloc[:,selected]

選択された特徴量の数: 6
[False False  True  True False False False  True  True False False False
 False False False  True False False False False False False False False
 False False False False False False False False  True]
Index(['mad_x', 'max_x', 'iqr_x', 'range_x', 'min_y', 'kurtosis_z'], dtype='object')


In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

random_state = 0

# X_train, X_test = X_selected[142:], X_selected[0:142]
# y_train, y_test = y[142:], y[0:142]

print(df1.shape, df4.shape)

df_train, df_test = pd.concat([df2, df3, df5, df6]), pd.concat([df1, df4])
X_train, X_test = df_train.iloc[:, :-5], df_test.iloc[:, :-5]
X_train, X_test = X_train.iloc[:,selected], X_test.iloc[:,selected]
y_train, y_test = df_train["label"].astype(int), df_test["label"].astype(int)

# データの標準化処理
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# kernel SVMのインスタンスを生成
model = SVC(kernel='rbf', random_state=random_state)

# モデルの学習
model.fit(X_train_std, y_train)

# トレーニングデータに対する精度
pred_train = model.predict(X_train_std)
accuracy_train = accuracy_score(y_train, pred_train)
precision_train = precision_score(y_train, pred_train)
recall_train = recall_score(y_train, pred_train)
print('トレーニングデータに対する正解率： %.2f' % accuracy_train)
print('トレーニングデータに対する適合率： %.2f' % precision_train)
print('トレーニングデータに対する再現率： %.2f' % recall_train)
cm = confusion_matrix(y_train, pred_train)
print(cm)

# テストデータに対する精度
pred_test = model.predict(X_test_std)
accuracy_test = accuracy_score(y_test, pred_test)
precision_test = precision_score(y_test, pred_test)
recall_test = recall_score(y_test, pred_test)
print('テストデータに対する正解率： %.2f' % accuracy_test)
print('テストデータに対する適合率： %.2f' % precision_test)
print('テストデータに対する再現率： %.2f' % recall_test)

cm = confusion_matrix(y_test, pred_test)
print(cm)

(31, 38) (81, 38)
トレーニングデータに対する正解率： 0.97
トレーニングデータに対する適合率： 0.95
トレーニングデータに対する再現率： 0.98
[[136   6]
 [  2 113]]
テストデータに対する正解率： 0.97
テストデータに対する適合率： 0.91
テストデータに対する再現率： 1.00
[[78  3]
 [ 0 31]]


In [10]:
def output_csv(
    base_df: pd.core.frame.DataFrame,
    pred_result: np.ndarray,
    columns: list=['frame_in', 'frame_out', 'label'],
    output_path: str="output_pred.csv"
):
    """
    base_df: predに対応するDataFrame
    pred_result: 判別後の予測結果 (ndarray)
    columns: outputするカラム名のリスト
    """
    base_df = base_df[columns]
    base_df = base_df.assign(pred=pred_result)
    
    frame_in = base_df["frame_in"].iloc[0].astype(int)
    frame_out = base_df["frame_out"].iloc[-1].astype(int)
    cum_pred = np.zeros(frame_out - frame_in)

    out_df = pd.DataFrame(columns=['frame', 'label', 'pred'])

    for i, (_, row) in enumerate(base_df.iterrows()):
        fin = row['frame_in'].astype(int)
        fout = row['frame_out'].astype(int)

        for frame in range(fin, fout + 1):
            if (out_df['frame'] == frame).sum() == 0:
                out_df = out_df.append({
                    'frame': frame,
                    'label': row['label'],
                    'pred': row['pred'],
                }, ignore_index=True)
            else:
                if row['pred'] == 1:
                    out_df.loc[out_df['frame'] == frame, 'pred'] += 1
                    
    if output_path is not None:
        out_df.to_csv(output_path, index=False)

    return out_df


test_df0 = df1.copy()
pred_test0 = pred_test[0:31]
test_df1 = df4.copy()
pred_test1 = pred_test[31:]
    
out_df0 = output_csv(test_df0, pred_test0, output_path="output_pred0.csv")
out_df1 = output_csv(test_df1, pred_test1, output_path="output_pred1.csv")
print(out_df0)
print((out_df0['pred'] == 2).sum())
print((out_df1['pred'] == 2).sum())

     frame  label  pred
0      1.0    1.0   1.0
1      2.0    1.0   1.0
2      3.0    1.0   1.0
3      4.0    1.0   1.0
4      5.0    1.0   1.0
..     ...    ...   ...
523  556.0    1.0   1.0
524  557.0    1.0   1.0
525  558.0    1.0   1.0
526  559.0    1.0   1.0
527  560.0    1.0   1.0

[528 rows x 3 columns]
464
16


In [28]:
import cv2
import tqdm
from typing import Tuple


def get_video_capture(video_path: str) -> cv2.VideoCapture:
    return cv2.VideoCapture(video_path)


def get_video_length(video_path: str) -> int:
    video_capture = get_video_capture(video_path)
    return int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))


def get_frame_position(video_capture: cv2.VideoCapture) -> int:
    return int(video_capture.get(cv2.CAP_PROP_POS_FRAMES))


def set_frame_position(video_capture: cv2.VideoCapture, position: int) -> int:
    return int(video_capture.set(cv2.CAP_PROP_POS_FRAMES, position))


def get_video_dimension(video_path: str) -> Tuple[int, int]:
    video_capture = get_video_capture(video_path)
    return int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))


def get_video_framerate(video_path: str) -> float:
    video_capture = get_video_capture(video_path)
    return float(video_capture.get(cv2.CAP_PROP_FPS))


def output_video_nod(nod_df, video_path: str, output_path: str = "output_video_nod"):
    video_capture = get_video_capture(video_path)
    video_length = get_video_length(video_path)

    print(nod_df)
    start, end = nod_df['frame'].iloc[0].astype(int), nod_df['frame'].iloc[-1].astype(int)
#     bar = tqdm.tqdm(total=end - start)

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    video_writer = cv2.VideoWriter(output_path, fourcc, get_video_framerate(video_path),
                                   get_video_dimension(video_path))

    frame_index = start
    set_frame_position(video_capture, start)  # Move position
    while video_capture.isOpened() and get_frame_position(video_capture) in range(start, end+1):
        
#         print(frame_index, nod_df.shape)
        print(nod_df[nod_df['frame'] == frame_index]['pred'])
        
#         if frame_index > nod_df.shape[0]:
#             break
        
        ret, frame = video_capture.read()
#         # Progress
#         bar.update(1)
#         bar.refresh()

        nod = nod_df[nod_df['frame'] == frame_index]['pred'].values[0]

        # BBox
        left, top = 200, 200
        right, bottom = 400, 400
        p1 = (left, top)
        p2 = (right, bottom)
        texts = {0: "No", 1: "Nod", 2: "Nod!!!"}

        # Draw circle when nod flag == 1 or 2
        if nod > 0:
            mid = (np.array(p1) + np.array(p2)) // 2
#             frame = cv2.circle(frame, tuple(mid), radius=100, color=(0, 0, 255), thickness=int(1 * nod))


            frame = cv2.putText(frame, texts[nod], (50, 50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 255), 5, cv2.LINE_AA)

        video_writer.write(frame)
        frame_index += 1

    video_capture.release()
    video_writer.release()

nod_df = out_df0.copy()
video_path = "../../movie/processed_data/nod2-1.avi"
output_path = "pred_nod2-1.mp4"
output_video_nod(nod_df, video_path, output_path)

print("finished")

# nod_df = out_df1
# video_path = "../../movie/processed_data/other2-1.avi"
# output_path = "pred_other2-1.mp4"
# output_video_nod(nod_df, video_path, output_path)

     frame  label  pred
0      1.0    1.0   1.0
1      2.0    1.0   1.0
2      3.0    1.0   1.0
3      4.0    1.0   1.0
4      5.0    1.0   1.0
..     ...    ...   ...
523  556.0    1.0   1.0
524  557.0    1.0   1.0
525  558.0    1.0   1.0
526  559.0    1.0   1.0
527  560.0    1.0   1.0

[528 rows x 3 columns]
0    1.0
Name: pred, dtype: float64
1    1.0
Name: pred, dtype: float64
2    1.0
Name: pred, dtype: float64
3    1.0
Name: pred, dtype: float64
4    1.0
Name: pred, dtype: float64
5    1.0
Name: pred, dtype: float64
6    1.0
Name: pred, dtype: float64
7    1.0
Name: pred, dtype: float64
8    1.0
Name: pred, dtype: float64
9    1.0
Name: pred, dtype: float64
10    1.0
Name: pred, dtype: float64
11    1.0
Name: pred, dtype: float64
12    1.0
Name: pred, dtype: float64
13    1.0
Name: pred, dtype: float64
14    1.0
Name: pred, dtype: float64
15    1.0
Name: pred, dtype: float64
16    2.0
Name: pred, dtype: float64
17    2.0
Name: pred, dtype: float64
18    2.0
Name: pred, dtype: flo

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
pwd

In [ ]:
# 各動画の開始行indexを取得
vid_index_list = df.index[df['timestamp_in'] == 0].tolist()
print(vid_index_list)

result0_pred = pred_test[vid_index_list[0] : vid_index_list[1]]
result0_df = df.iloc[vid_index_list[0]:vid_index_list[1]]
# result0_df.iloc[0]
framein = result0_df["frame_in"].iloc[0].astype(int)
frameout = result0_df["frame_out"].iloc[-1].astype(int)

result0 = np.zeros(frameout - framein)

for i, (_, data) in enumerate(result0_df.iterrows()):
    pred = result0_pred[i]

    if pred == 1:
        fin = data["frame_in"].astype(int)
        fout = data["frame_out"].astype(int)
        result0[fin:fout] += 1

import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(result0)
result0